In [109]:
import pandas as pd
import numpy as np
import os
import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/IW_student/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [110]:
long_ingr=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_ingr_ge_003.csv',index_col=0)
long_ingr.shape

(1849, 5)

In [111]:
long_ingr[long_ingr['recipe_id']==27351]

,recipe_id,recipe_href,name,ingredient_name,en_ingr
1821,27351,/rezept/437163/Haehnchen-mit-scharfem-Paprikak...,"gutgeräuchert auf dem bildpaprikaknacker,falls...",gutgeräuchert auf dem bildpaprikaknacker,wellsmoked picture pepper cracker


In [3]:
long_ingr.drop('en_ingr',axis=1,inplace=True)

In [4]:
long_ingr=long_ingr.assign(ingredient_name=long_ingr['ingredient_name'].str.split(',')).explode('ingredient_name')

In [5]:
# load data frome here!!!
long_ingr.reset_index(drop=True,inplace=True)

In [6]:
long_three_ingr=long_ingr[long_ingr['ingredient_name'].str.split().str.len()==3]
long_three_ingr.shape

(153, 4)

In [7]:
long_three_ingr

,recipe_id,recipe_href,name,ingredient_name
20,92629,/rezept/90278/Nudel-Kaese-Salat-mit-Coktailtom...,salatgenuss extra leichtz.b. von du darfst,salatgenuss extra leicht
29,47970,/rezept/110769/Roter-Gefluegelsalat.html,brathähnchen fleisch mithaut tiefgefroren,brathähnchen fleisch tiefgefroren
38,42365,/rezept/402197/ROOIE-ZALM.html,pfeffer muskatnuss aus dermühle schwarz,pfeffer muskatnuss schwarz
52,54530,/rezept/378323/Jaegerpfanne.html,suppenwürfel - klare suppemit suppengrün,klare suppemit suppengrün
78,98278,/rezept/465055/Gelbe-Bohnen-mit-Mehlnockerln.html,"gelbe breite bohnen -tk,blanchiert",gelbe breite bohnen
...,...,...,...,...
1847,15875,/rezept/435661/Ofenspargel-mit-Speck-an-karame...,wie der ofenspargel - nurohne salz,wie der ofenspargel
1857,86451,/rezept/308177/Zuercher-Geschnetzeltes-mit-Roe...,kalbfleisch z.b nuss in ca.1 cm breite streife...,kalbfleisch z.b nuss
1859,94041,/rezept/54351/Gyros-Suppe.html,zwiebelsuppe von maggimeiserklasse für 3/4 ltr...,zwiebelsuppe von maggimeiserklasse
1889,13559,/rezept/451671/Huehnerbrustragout-mit-Champign...,reis mit gebratenenreisnudeln - rezept inmeine...,reis mit gebratenenreisnudeln


In [8]:
long_three_ingrs=list(set(long_three_ingr['ingredient_name'].values.tolist()))
len(long_three_ingrs)

150

In [9]:
long_three_ingrs.sort()
long_three_ingrs

['1 tüteentspricht 5',
 '10 stueckohe haut',
 '2 tl paprikapulver',
 '200gr. bergkäse untermischen',
 'african zulu dipp',
 'amerikan cookies dunkel',
 'back trenn spray',
 'backzeit im umluftofen',
 'basilikumsauce mitcrème fraîche',
 'batida de coco',
 'bei anderenkürbisarten siehebeschreibung',
 'blattgelantiene 6 blatt',
 'bouillon 2 huhn',
 'bouillon pur  knorr',
 'brathähnchen fleisch  frisch',
 'brathähnchen fleisch  tiefgefroren',
 'buco der sahnige',
 'candurin goldpulver hierhatte',
 'chili bitte nur',
 'chili gruen gehackt',
 'creme fraiche fürdeko',
 'dieäusseren blätter entfernt',
 'dr. oetker dessert',
 'dr. oetker götterspeisehimbeer',
 'dr. oetker paradiescreme',
 'dr. oetker puddingpulversahnegeschmack',
 'dr. oetker vanillesosse',
 'dr. oetker vanillin',
 'dunkler bisquitboden 2xgeschniten',
 'dunkler wienerboden 2xgeschnitten',
 'ein esslöffelweissen balsamico',
 'ein schuss hellerbalsamico',
 'eingelegter grüner pfeffer',
 'einige blätter liebstöckel',
 'einpaar spr

In [10]:
# total delete
delete_whole=['1 tüteentspricht 5','10 stueckohe haut','backzeit im umluftofen','bei anderenkürbisarten siehebeschreibung',
'frisch','tiefgefroren','candurin goldpulver hierhatte','dieäusseren blätter entfernt','fürerdbeerkonfitüre von dr.oetker',
'keramikschale mit heissemwasser','leber nach wahl...','tropfen rote farbe', 'um imeierschneider zerkleinernauch']

In [11]:
long_three_ingr=long_three_ingr[~long_three_ingr['ingredient_name'].isin(delete_whole)]

In [12]:
# word delete
delete_words=['2 tl','untermischen','200gr.','6 blatt','bitte','nur','gehackt','gruen','fürdeko','dr. oetker','einige blätter',
'einpaar','es geht','festes','flaschen guinnes','von knorr','frische','dicke','deutsche','instant','galetta',
'gefüllte','ganze','stifte','frisch','eigeneherstellung','ersatzweisegekaufte','auchschlachter','gepökelter',
'vomschwäbisch','geriebenen','geröstete','der mühle','gefroren','zbsp','hackepeterfertig',
'gewürzt','übrig','gedünstet','heissemwasser','kleine','deko','klösse','von','demgrün','getränk','69 kcal',
'420 gabtropfgewicht','umfeuchtigkeit','aufzunehmen','prise','von dr.oetker','vonz.b. knorr','frischvom','geflügelhof',
'reichlich','hautabziehen u.','gespritzt','überlassen','vom','hier','sagtman','extra','leicht','fixitalienische','art',
'gut','abgetropft','stichfest','ohneknochen','duroc','sind','auchhalbierte','für1','fridabella','nachgeschmack',
'spritzer','ider','gewürfelt','sorrynicht','gefüllt','siehe','foto','türkisches','geht','genauso','wie','der',
'a`250','gramm','1prise','bei bedarf','grob','gewürfelt','von maggimeiserklasse','tiefgefroren','gemahlener',
'gemahlen']

In [13]:
def del_words(ingredient):
    no_words=' '.join([c for c in ingredient.split() if c not in delete_words])
    return no_words

In [14]:
long_three_ingr['ingredient_name']=long_three_ingr['ingredient_name'].apply(lambda x:del_words(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
long_three_ingr.reset_index(drop=True,inplace=True)

In [16]:
# part delete
delete_parts=['2 tl','ein esslöffel','ein schuss helle','eingelegter','halbiert','grober','gekochten','mit je 2l','aus der mühle',
'gekörnte','6 blatt','69 kcal','420 gabtropfgewicht','bei bedarf','einige blätter','im internetkäuflich','eigeneherstellung!',
'nehme','dr.oetker','vonz.b. knorr','dr. oetker']

In [17]:
for part in delete_parts:
    for index, row in long_three_ingr.iterrows():
        if re.search(part,row.ingredient_name)!=None:
            long_three_ingr.loc[index,'ingredient_name']=re.sub(part,'',row.ingredient_name)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [18]:
long_three_ingr['ingredient_name']=long_three_ingr['ingredient_name'].str.strip()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
long_three_ingrs=list(set(long_three_ingr['ingredient_name'].values.tolist()))
len(long_three_ingrs)

137

In [20]:
long_three_ingrs.sort()
long_three_ingrs

['african zulu dipp',
 'amerikan cookies dunkel',
 'ammkoteletts',
 'anis',
 'back trenn spray',
 'basilikumsauce mitcrème fraîche',
 'batida de coco',
 'bergkäse',
 'blattgelantiene',
 'bouillon 2 huhn',
 'bouillon pur knorr',
 'brathähnchen fleisch',
 'buco sahnige',
 'bunter pfeffer',
 'chili',
 'chilliölnur',
 'cocktailtomaten',
 'creme fraiche',
 'dessert',
 'dunkler bisquitboden 2xgeschniten',
 'dunkler wienerboden 2xgeschnitten',
 'erdnussöl es',
 'essig weisswein essig',
 'flaschen guinnes irisch.bier',
 'fleischbrühe knorr',
 'frucht nuss schokolatetafeln',
 'gelbe breite bohnen',
 'gelbe tomaten',
 'gem. beeren himbeeren',
 'gemüse',
 'gemüsebrühpulver',
 'gemüsemischungcalifornische mitsonnenblumenkerne',
 'gemüsezwiebel',
 'germ od. oetkerdauerbackhefe',
 'ginger garlic paste',
 'gr blaue weintrauben',
 'grana padano parmesan',
 'grillgewürzdöner baharat',
 'grüne heringsfileets',
 'grüner pfeffer',
 'gulaschcreme',
 'gulaschfleisch rind',
 'götterspeisehimbeer',
 'herbe sa

In [21]:
%env GOOGLE_APPLICATION_CREDENTIALS =/home/IW_student/work/Ingredeints-translation-1d0aa41c5717.json

env: GOOGLE_APPLICATION_CREDENTIALS=/home/IW_student/work/Ingredeints-translation-1d0aa41c5717.json


In [22]:
from google.cloud import translate_v2 as translate
translate_client = translate.Client() # add environment varibale

In [23]:
def translation(ingredient):
    results=translate_client.translate(ingredient,target_language='en')
    return results['translatedText']

In [24]:
long_three_ingr['en_ingr']=long_three_ingr['ingredient_name'].apply(lambda x:translation(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
long_three_ingr

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,92629,/rezept/90278/Nudel-Kaese-Salat-mit-Coktailtom...,salatgenuss extra leichtz.b. von du darfst,salatgenuss,salad enjoyment
1,47970,/rezept/110769/Roter-Gefluegelsalat.html,brathähnchen fleisch mithaut tiefgefroren,brathähnchen fleisch,fried chicken meat
2,42365,/rezept/402197/ROOIE-ZALM.html,pfeffer muskatnuss aus dermühle schwarz,pfeffer muskatnuss schwarz,pepper nutmeg black
3,54530,/rezept/378323/Jaegerpfanne.html,suppenwürfel - klare suppemit suppengrün,klare suppemit suppengrün,clear suppemit suppengrün
4,98278,/rezept/465055/Gelbe-Bohnen-mit-Mehlnockerln.html,"gelbe breite bohnen -tk,blanchiert",gelbe breite bohnen,yellow broad beans
...,...,...,...,...,...
136,106216,/rezept/157015/Kebab-Salat.html,türkisches grillgewürz%x22döner baharat%x22 - ...,grillgewürzdöner baharat,grillgewürz
137,15875,/rezept/435661/Ofenspargel-mit-Speck-an-karame...,wie der ofenspargel - nurohne salz,ofenspargel,baked asparagus
138,86451,/rezept/308177/Zuercher-Geschnetzeltes-mit-Roe...,kalbfleisch z.b nuss in ca.1 cm breite streife...,kalbfleisch z.b nuss,veal e.g. nut
139,94041,/rezept/54351/Gyros-Suppe.html,zwiebelsuppe von maggimeiserklasse für 3/4 ltr...,zwiebelsuppe maggimeiserklasse,onion soup maggimier class


In [26]:
long_three_ingr_en=list(set(long_three_ingr['en_ingr'].values.tolist()))
len(long_three_ingr_en)

136

In [27]:
long_three_ingr_en.sort()
long_three_ingr_en

['Dark biscuit base cut twice',
 'Red pepper',
 'according to berries raspberries',
 'african zulu dipp',
 'american cookies dark',
 'ammkoteletts',
 'anis',
 'back trenn spray',
 'baked asparagus',
 'basilica, fresh mitcream sauce',
 'basketmehl',
 'beef broth with meat',
 'beef shoulder or wad ham',
 'black pepper broth',
 'bottles of guinnes irish beer',
 'bouillon 2 chicken',
 'carrots',
 'catfish fillet alternatively other firm fleshy fish fillet',
 'cheese dutch',
 'chicken breast',
 'chili',
 'chilliölnur',
 'chocolate kisses for',
 'clear suppemit suppengrün',
 'cocktailtomaten',
 'coconut beat',
 'colorful pepper',
 'cover bright dark',
 'cremfine branch 7%',
 'dark wiener floor 2x cut',
 'dessert',
 'fried chicken meat',
 'fruit nut chocolate bars',
 'garlic bulbs with',
 'gemüsemischungcalifornische mitsonnenblumenkerne',
 'germ or oetker permanent yeast',
 'ginger garlic paste',
 'gingerbread with chocolate icing',
 'goulash beef',
 'goulash cream',
 'gr blue grapes',
 'gre

In [30]:
# whole delete
whole_delete=['cover bright dark','sahnige hole','two bears sack',]

In [47]:
long_three_ingr=long_three_ingr[~long_three_ingr['en_ingr'].isin(whole_delete)]

In [31]:
# part delete
delete_parts=['according to','american','dark','baked','with meat','or wad ham','bottles of guinnes irish','alternatively other firm fleshy fish fillet',
'fillet','clear suppemit','2x cut','bulbs with','germ or oetker permanent','gr blue','n. b','or clarified butter',
'from mill','mill','to remove skin u.','two bears sack']

In [32]:
long_three_ingr.reset_index(drop=True,inplace=True)

In [33]:
for part in delete_parts:
    for index, row in long_three_ingr.iterrows():
        if re.search(part,row.en_ingr)!=None:
            long_three_ingr.loc[index,'en_ingr']=re.sub(part,'',row.en_ingr)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [34]:
# word delete
delete_words=['for','colorful','7%','fillets','maggi','knorr','it','pockets','powder','pure','enjoyment']

In [35]:
def del_words(ingredient):
    no_words=' '.join([c for c in ingredient.split() if c not in delete_words])
    return no_words

In [36]:
long_three_ingr['en_ingr']=long_three_ingr['en_ingr'].apply(lambda x:del_words(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
long_three_ingr_en=list(set(long_three_ingr['en_ingr'].values.tolist()))
len(long_three_ingr_en)

107

In [80]:
long_three_ingr_en.sort()
long_three_ingr_en

['',
 'Batida_de_coco',
 'Cremefine',
 'Paradiescreme',
 'Teriyaki marinade',
 'Wiener_tortenbode',
 'african zulu dipp',
 'almond',
 'anis',
 'asparagus',
 'balsamic',
 'basil_sauce,cream_sauce',
 'beef',
 'beer',
 'bell pepper',
 'bell_pepper',
 'berry',
 'biscuit',
 'black pepper',
 'bouillon',
 'broad bean',
 'broth',
 'cabbage',
 'carrots',
 'catfish',
 'cheese',
 'cherry',
 'chicken',
 'chicken broth',
 'chili',
 'chili oil',
 'chocolate',
 'cocktail tomato',
 'cod',
 'cookies',
 'cream',
 'cream cheese',
 'cream_pudding_powder',
 'curry',
 'dessert',
 'dutch cheese',
 'fish',
 'flour',
 'garlic',
 'gelatin',
 'ginger',
 'ginger garlic paste',
 'goulash cream',
 'grape juice',
 'grapes',
 'herb salt',
 'herring',
 'jam',
 'lovage',
 'milk,coffee',
 'mountain cheese',
 'mutton',
 'nougat,hazelnuts',
 'oil',
 'onion',
 'onion soup',
 'oriental spice',
 'pak choy',
 'paprika',
 'peanut oil',
 'pepper',
 'pepper paste',
 'pepper,nutmeg',
 'pizza dough',
 'pork',
 'potato',
 'potato,c

In [78]:
long_three_ingr['en_ingr'][long_three_ingr['en_ingr']=='meat broth']='broth'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [72]:
# change_dict
change_dict={'balsamic vinegar':'balsamic','basilica, fresh mitcream sauce':'basil_sauce,cream_sauce',
             'berries raspberries':'berry','chicken bouillon':'chicken broth','coconut beat':'Batida_de_coco',
             'cover bright':'','radish rose with piri':'radish,piri','red groats':'','salatsosse':'salat_sauce',
             'salt water with sugar':'salt,water,sugar','soup white wine':'white wine','suppengrün':'vegetable',
             'toast sausages':'sausages','veal e.g. nut':'veal','vegetable onion':'onion','vegetable stock':'vegetable',
             'wiener floor':'Wiener_tortenbode',
             'ammkoteletts':'mutton','basketmehl':'flour','black pepper broth':'black pepper','bouillon 2 chicken':'chicken bouillon',
 'cheese dutch':'dutch cheese','chicken breast':'chicken','chilliölnur':'chili oil','chocolate kisses':'chocolate',
 'clear broth':'broth','cocktailtomaten':'cocktail tomato','cremfiranch':'Cremefine','fried chicken meat':'chicken',
'fruit nut chocolate bars':'chocolate','gemüsemischungcalifornische mitsonnenblumenkerne':'vegetable,sunflower seed',
'gingerbread with chocolate icing':'ginger','green herring s':'herring','green pepper':'bell_pepper',
'grillgewürz':'spice','kräutersalz':'seasoned salt','jam jam jams':'jam','kikkoman teriyaki marinade':'Teriyaki marinade',
'leaf gelantiene':'gelatin','milk with coffee':'milk,coffee','mixed vegetables':'vegetables',
'onion soup maggimier class':'onion soup','paprika green ...':'bell pepper','paradiescreme':'Paradiescreme',
'parmesan grana padano':'cheese','parmesanod. other hard cheese':'cheese','pepper nutmeg black':'pepper,nutmeg',
'philadelphia kaese':'cream cheese','pig leg':'pork','pizza dough with sauce':'pizza dough',
'pok choi choy':'pak choy','potato with cream cheese':'potato,cream cheese','prik kee noo':'chili',
'puddingpulversahnegeschmack':'cream_pudding_powder','rbalsamic':'balsamic vinegar','recipe pizza base dough':'pizza dough',
'red grape juice':'grape juice','red thai curry':'curry','red wine vinegar flavored with mitestragon':'red wine vinegar',
 'rice with fried rice noodles':'rice,rice noodle','roast ham':'roasted ham','saiten würstchen':'sausage',
 'scene':'cream','sossenpulver vanilla':'vanilla_sauce','soup chicken meat':'chicken','sour cherry':'cherry',
'spanischerunzelkartoffeln – p':'potato','spice oriental':'oriental spice','sugar salt':'sugar,salt',
 'sweet almond acc.':'almond','tart cream chocolate':'chocolate','tomato sauce with herbs':'tomato sauce,herbs',
 'tortellini with spinach':'tortellini,spinach','turkish pepper paste':'pepper paste','turmeric spice':'turmeric',
 'vanillepudding':'vanilla_pudding','vanillesosse':'vanilla_sauce','vanillin':'vanilla',
 'vinegar white wine vinegar':'white wine vinegar','water or chicken broth':'chicken broth',
 'weisskohl':'cabbage','white balsamic':'balsamic vinegar','white fish':'fish','white nougat with hazelnuts':'nougat,hazelnuts',
 'winter cod':'cod','with garlic':'garlic','wort liquid':'spice',
            'Dark biscuit base cut twice':'biscuit','Red pepper':'bell_pepper','back trenn spray':'',
            'yellow broad beans':'broad bean','yellow tomatoes':'tomato'}

In [73]:
long_three_ingr.reset_index(drop=True,inplace=True)

In [74]:
for k,v in change_dict.items():
    for index,row in long_three_ingr.iterrows():
        if re.search(k,row.en_ingr)!=None:
            long_three_ingr.loc[index,'en_ingr']=v

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [83]:
long_three_ingr['en_ingr'][long_three_ingr['en_ingr']=='paprika']='chili'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [ ]:
# contain
# 'beef','pork','turkey',

In [75]:
long_three_ingr.loc[long_three_ingr['en_ingr'].str.contains('beef'),'en_ingr']='beef'
long_three_ingr.loc[long_three_ingr['en_ingr'].str.contains('pork'),'en_ingr']='pork'
long_three_ingr.loc[long_three_ingr['en_ingr'].str.contains('turkey'),'en_ingr']='turkey'

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [86]:
long_three_ingr[long_three_ingr['en_ingr']=='pepper paste']

,recipe_id,recipe_href,name,ingredient_name,en_ingr
55,2154,/rezept/376933/Scharfe-Nudel-Gemuese-Pfanne-mi...,aci biber - türkischepaprikapaste siehe foto,türkischepaprikapaste,pepper paste


In [90]:
long_three_ingr_en=list(set(long_three_ingr['en_ingr'].values.tolist()))
len(long_three_ingr_en)

111

In [91]:
long_three_ingr_en.sort()
long_three_ingr_en

['Batida_de_coco',
 'Cremefine',
 'Paradiescreme',
 'Teriyaki marinade',
 'Wiener_tortenbode',
 'african zulu dipp',
 'almond',
 'anis',
 'asparagus',
 'balsamic',
 'basil_sauce',
 'beef',
 'beer',
 'bell pepper',
 'bell_pepper',
 'berry',
 'biscuit',
 'black pepper',
 'bouillon',
 'broad bean',
 'broth',
 'cabbage',
 'carrots',
 'catfish',
 'cheese',
 'cherry',
 'chicken',
 'chicken broth',
 'chili',
 'chili oil',
 'chocolate',
 'cocktail tomato',
 'cod',
 'coffee',
 'cookies',
 'cream',
 'cream cheese',
 'cream_pudding_powder',
 'cream_sauce',
 'curry',
 'dessert',
 'dutch cheese',
 'fish',
 'flour',
 'garlic',
 'gelatin',
 'ginger',
 'ginger garlic paste',
 'goulash cream',
 'grape juice',
 'grapes',
 'hazelnuts',
 'herb salt',
 'herbs',
 'herring',
 'jam',
 'lovage',
 'milk',
 'mountain cheese',
 'mutton',
 'nougat',
 'nutmeg',
 'oil',
 'onion',
 'onion soup',
 'oriental spice',
 'pak choy',
 'peanut oil',
 'pepper',
 'pepper paste',
 'piri',
 'pizza dough',
 'pork',
 'potato',
 'p

In [87]:
long_three_ingr=long_three_ingr[long_three_ingr['en_ingr']!='']

In [89]:
long_three_ingr=long_three_ingr.assign(en_ingr=long_three_ingr['en_ingr'].str.split(',')).explode('en_ingr')

In [92]:
lemmatizer = WordNetLemmatizer()

In [93]:
long_three_ingr['tag']='n'

In [94]:
long_three_ingr['ingr_lemma']=long_three_ingr.apply(lambda x:lemmatizer.lemmatize(x.en_ingr,x.tag),axis=1)

In [97]:
long_three_ingr['ingr_lemma'][long_three_ingr['ingr_lemma']=='ani']='anise'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [106]:
long_three_ingr_en=list(set(long_three_ingr['ingr_lemma'].values.tolist()))
len(long_three_ingr_en)

106

In [107]:
long_three_ingr_en.sort()
long_three_ingr_en

['Batida_de_coco',
 'Cremefine',
 'Paradiescreme',
 'Teriyaki_marinade',
 'Wiener_tortenbode',
 'african_zulu_dipp',
 'almond',
 'anise',
 'asparagus',
 'balsamic',
 'basil_sauce',
 'beef',
 'beer',
 'bell_pepper',
 'berry',
 'biscuit',
 'black_pepper',
 'bouillon',
 'broad_bean',
 'broth',
 'cabbage',
 'carrot',
 'catfish',
 'cheese',
 'cherry',
 'chicken',
 'chicken_broth',
 'chili',
 'chili_oil',
 'chocolate',
 'cocktail_tomato',
 'cod',
 'coffee',
 'cooky',
 'cream',
 'cream_cheese',
 'cream_pudding_powder',
 'cream_sauce',
 'curry',
 'dessert',
 'dutch_cheese',
 'fish',
 'flour',
 'garlic',
 'gelatin',
 'ginger',
 'ginger_garlic_paste',
 'goulash_cream',
 'grape',
 'grape_juice',
 'hazelnut',
 'herb',
 'herb_salt',
 'herring',
 'jam',
 'lovage',
 'milk',
 'mountain_cheese',
 'mutton',
 'nougat',
 'nutmeg',
 'oil',
 'onion',
 'onion_soup',
 'oriental_spice',
 'pak_choy',
 'peanut_oil',
 'pepper',
 'pepper_paste',
 'piri',
 'pizza_dough',
 'pork',
 'potato',
 'pudding',
 'radish',
 

In [100]:
long_three_ingr['ingr_lemma']=long_three_ingr['ingr_lemma'].apply(lambda x:x.replace(' ','_'))

In [105]:
long_three_ingr.drop('tag',axis=1,inplace=True)
long_three_ingr.reset_index(drop=True,inplace=True)

In [108]:
long_three_ingr.to_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_three_ingr_done.csv')